<a href="https://colab.research.google.com/github/farieu/data-analysis/blob/Pipeline/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline de Pré-Processamento

Importando bibliotecas necessárias para criar o Pipeline

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Binarizer, LabelEncoder, MultiLabelBinarizer

O pipeline deve tratar o dataset na sua forma original, então vou estar lendo o .csv do arquivo do Kaggle.

In [ ]:
# Dataset 'cru', sem nenhuma limpeza/tratamento (original do Kaggle)
df = pd.read_csv('/content/drive/MyDrive/BackEnd/GoodReads_100k_books.csv')


      Pipeline de pré-processamento do Dataframe, com parâmetros flexíveis para permitir o uso em diferentes colunas e cenários.

        Parâmetros:
        - to_drop: lista de colunas a serem removidas do DataFrame.
        - to_binarize: lista de colunas para aplicar codificação de rótulos (LabelEncoder).
        - to_mtbinarize: lista de colunas a serem binarizadas com o MultiLabelBinarizer, útil para dados multiclasse.
        - drop_nan: flag para remover linhas com valores ausentes em qualquer coluna (padrão: True).
        - drop_duplicates: flag para remover linhas duplicadas do DataFrame (padrão: True).
        
        
        

In [ ]:
class DataFramePreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, to_drop=None, to_binarize=None, to_mtbinarize=None, drop_nan=True, drop_duplicates=True):
        self.to_drop = to_drop if to_drop else []
        self.to_binarize = to_binarize if to_binarize else []
        self.to_mtbinarize = to_mtbinarize if to_mtbinarize else []
        self.drop_nan = drop_nan
        self.drop_duplicates = drop_duplicates
        self.label_encoder = LabelEncoder()
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        return self

    def transform(self, dataframe):
        if self.drop_duplicates:
            dataframe = dataframe.drop_duplicates()

        for column in self.to_drop:
            if column in dataframe.columns:
                dataframe = dataframe.drop(column, axis=1)

        if self.drop_nan:
            dataframe = dataframe.dropna()

        for column in self.to_binarize:
            if column in dataframe.columns:
                dataframe[column] = self.label_encoder.fit_transform(dataframe[column])

        for column in self.to_mtbinarize:
            if column in dataframe.columns:
                dataframe[column] = dataframe[column].apply(lambda x: x.split(',') if isinstance(x, str) else [])
                column_encoded = self.mlb.fit_transform(dataframe[column])
                column_df = pd.DataFrame(column_encoded, columns=self.mlb.classes_, index=dataframe.index)
                dataframe = pd.concat([dataframe.drop(columns=[column]), column_df], axis=1)

        return dataframe


cleaning_pipeline = Pipeline([
    ('cleaner', DataFramePreprocessor(
        to_drop=['isbn', 'isbn13', 'link', 'img'],
        to_binarize=['author', 'bookformat', 'title'],
        to_mtbinarize=['genre']
    ))
])
df_cleaned = cleaning_pipeline.fit_transform(df)

(84054, 1187)

O pipeline foi montado seguindo o fluxo que eu normalmente utilizei ao implementar o código de pré-processamento, adicionei alguns parâmetros para que ficasse mais flexível para caso eu quisesse realizar o mesmo processo para outras colunas, se necessário.

Chamando o pipeline para tratar o dataframe e armazenar a nova versão na variável **df_cleaned**.

In [ ]:
df_cleaned = cleaning_pipeline.fit_transform(df)

In [13]:
df_cleaned_shape
# Mesmas dimensões do dataset que eu tratei manualmente.

(84054, 1187)

In [12]:
df_cleaned.head()

,author,bookformat,desc,pages,rating,reviews,title,totalratings,10th Century,11th Century,...,Young Adult Paranormal,Young Adult Romance,Young Adult Science Fiction,Young Readers,Yuri,Zambia,Zen,Zimbabwe,Zombies,æ¼«ç”»
0,32032,56,Reveals that several hundred thousand Indians ...,0,3.52,5,8698,33,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8446,100,Fashion Sourcebook - 1920s is the first book i...,576,4.51,6,21651,41,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2789,100,The seminal history and analysis of the Hungar...,124,4.15,2,28925,26,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7378,56,"""All-American Anarchist"" chronicles the life a...",324,3.83,1,4505,6,0,0,...,0,0,0,0,0,0,0,0,0,0
5,24524,56,Why is common sense so uncommon when it comes ...,368,3.73,7,63417,119,0,0,...,0,0,0,0,0,0,0,0,0,0


Como a binarização é a parte que eu ajusto o parâmetro de modelo, acabei não colocando-o na parte do pipeline, só colocando-o no conjunto de teste antes de treinar o modelo.